In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType

In [99]:
from pyspark.sql.window import Window as w

In [3]:
spark = SparkSession.builder.appName('Renewal Energy').getOrCreate()

In [54]:
schema = StructType([\
    StructField('COUNTRY CODE', StringType(), True),\
    StructField('INDICATOR', StringType(), True),\
    StructField('SUBJECT', StringType(), True),\
    StructField('MEASURE', StringType(), True),\
    StructField('FREQUENCY', StringType(), True),\
    StructField('YEAR', IntegerType(), True),\
    StructField('Value', FloatType(), True),\
    StructField('Flag Codes', StringType(), True)                    
])

In [55]:
df = spark.read.schema(schema).csv('renewable_energy.csv', header = True, inferSchema = True)

In [56]:
df.printSchema()

root
 |-- COUNTRY CODE: string (nullable = true)
 |-- INDICATOR: string (nullable = true)
 |-- SUBJECT: string (nullable = true)
 |-- MEASURE: string (nullable = true)
 |-- FREQUENCY: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- Value: float (nullable = true)
 |-- Flag Codes: string (nullable = true)



In [58]:
df.describe().show()

+-------+------------+---------+-------+----------------+---------+------------------+-----------------+----------+
|summary|COUNTRY CODE|INDICATOR|SUBJECT|         MEASURE|FREQUENCY|              YEAR|            Value|Flag Codes|
+-------+------------+---------+-------+----------------+---------+------------------+-----------------+----------+
|  count|       15904|    15904|  15904|           15904|    15904|             15904|            12017|      3887|
|   mean|        NULL|     NULL|   NULL|            NULL|     NULL|            1987.5|13221.08363322194|      NULL|
| stddev|        NULL|     NULL|   NULL|            NULL|     NULL|16.163738049041793|92035.01903133834|      NULL|
|    min|         AGO|RENEWABLE|    TOT|            KTOE|        A|              1960|              0.0|         L|
|    max|         ZWE|RENEWABLE|    TOT|PC_PRYENRGSUPPLY|        A|              2015|        1894018.6|         L|
+-------+------------+---------+-------+----------------+---------+-----

In [197]:
f'Total Number of Rows: {df.count()}'

'Total Number of Rows: 12017'

In [176]:
df = df.withColumn('RANK', func.rank().over(w.partitionBy('COUNTRY CODE').orderBy(df['YEAR'].desc())))

__TOTAL PRIMARY ENERGY SUPPLY AMOUNTS FOR MOST RECENT YEAR PER COUNTRY__

In [198]:
df.select(\
         'COUNTRY CODE',\
          df['YEAR'].alias('MOST RECENT YEAR'),\
          df.Value.alias('TOTAL PRIMARY ENERGY SUPPLY AMOUNT')\
         )\
.filter(df['Value'].isNotNull())\
.filter(df.MEASURE == 'PC_PRYENRGSUPPLY')\
.filter(df['RANK'] == 1)\
.show()

+------------+----------------+----------------------------------+
|COUNTRY CODE|MOST RECENT YEAR|TOTAL PRIMARY ENERGY SUPPLY AMOUNT|
+------------+----------------+----------------------------------+
|         AGO|            2014|                             51.72|
|         ALB|            2014|                             27.29|
|         ARE|            2014|                              0.18|
|         ARG|            2014|                              8.75|
|         ARM|            2014|                              6.95|
|         AUS|            2015|                              6.54|
|         AUT|            2015|                             29.21|
|         AZE|            2014|                              1.63|
|         BEL|            2015|                              6.28|
|         BEN|            2014|                             53.41|
|         BGD|            2014|                             26.23|
|         BGR|            2014|                              9

__KTOE AMOUNTS FOR MOST RECENT YEAR PER COUNTRY__

In [172]:
df.select(\
         'COUNTRY CODE',\
          df['YEAR'].alias('MOST RECENT YEAR'),\
          func.round(df.Value, 2).alias('KTOE Amounts')\
         )\
.filter(df['Value'].isNotNull())\
.filter(df.MEASURE == 'KTOE')\
.filter(df['RANK'] == 1)\
.show()

+------------+----------------+------------+
|COUNTRY CODE|MOST RECENT YEAR|KTOE Amounts|
+------------+----------------+------------+
|         AGO|            2014|     7588.81|
|         ALB|            2014|      637.31|
|         ARE|            2014|      124.94|
|         ARG|            2014|     7576.15|
|         ARM|            2014|      205.54|
|         AUS|            2015|     8544.65|
|         AUT|            2015|     9594.93|
|         AZE|            2014|      234.04|
|         BEL|            2015|     3321.58|
|         BEN|            2014|      2289.9|
|         BGD|            2014|     9291.88|
|         BGR|            2014|     1784.59|
|         BHR|            2014|         0.0|
|         BIH|            2014|     2002.57|
|         BLR|            2014|     1443.97|
|         BOL|            2014|      1233.4|
|         BRA|            2014|   116986.84|
|         BRN|            2014|        0.17|
|         BWA|            2014|      542.78|
|         

In [200]:
spark.stop()